In [ ]:
# Importing the required libraries, custom libraries and setting the python path
from load_env_and_set_pythonpath import print_pythonpath
import os
import inversion_utils as iu
import helita_io_lp as lp
import time
import numpy as np
from skimage.transform import resize
import ambig_utils as au
from sunpy.coordinates import sun
from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.coordinates import frames
import numpy as np
if au.is_notebook():
    from IPython.display import clear_output
print_pythonpath()

In [ ]:
import importlib
importlib.reload(iu)

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')
# Check the input configuration
config = iu.check_input_config(input_config, pprint=True, confirm=False)

In [ ]:
# Load the configuration from the JSON file

# Path to ambig executable and par file
ambig_executable_path = config['ambig_executable_path']
ambig_par = config['ambig_par']

# Fcubes file names saved in save_dir directory
ambig_input_dir = config['ambig_input_dir']
fbazi = config['fbazi']
fbhor = config['fbhor']
fblos = config['fblos']
ambig_save_dir = config['ambig_save_dir']

# Cropping and resize details
crop = config['crop']
xorg = config['xorg']
xsize = config['xsize'] 
yorg = config['yorg']
ysize = config['ysize']
rescale = config['rescale']

# North up and field of view angle details
is_north_up = config['is_north_up']
fov_angle = config['fov_angle']

# Fits file details from CRISP header
fits_info = config['fits_info']
pix = config['scale']

# Verbosity in prints
verbose = config['verbose']

In [ ]:
# Create a unique prefix for the output files based on the current time
id = int(time.time())
prefix = '_'.join(fblos.split('_')[1:-2]) + '_' + str(id) + '_'
print(f'prefix: {prefix}')

In [ ]:
# Load the data from the saved files
# data_dir = '/mn/stornext/d20/RoCS/aditi/project/sst/inversion/pyMilne/ITA/temp3/'
blos = lp.getdata(os.path.join(ambig_input_dir, fblos)) # nx, ny, nt
bhor = lp.getdata(os.path.join(ambig_input_dir, fbhor))
bazi = lp.getdata(os.path.join(ambig_input_dir, fbazi))

In [ ]:
# Obtain the time index from the file name
time_index = int(fblos.split('_')[-1].split('-')[0])
print(f'time index: {time_index}')

time_index = config['time_range'][0]
print(f'time index: {time_index}')

tt = time_index

In [ ]:
blos.shape[2]

In [ ]:
tt = 35

In [ ]:
# Get the range of latitudes and longitudes from the fits file info

x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
xcen = (x1 + x2) / 2
ycen = (y1 + y2) / 2
tobs = fits_info['all_start_times'][tt]

if blos.shape[2] == 1: # only one time step is loaded
# drop the redundant time axis
    blos = blos.squeeze().T # ny, nx
    bhor = bhor.squeeze().T
    bazi = bazi.squeeze().T
else:
    blos = blos[:,:,tt].T
    bhor = bhor[:,:,tt].T
    bazi = bazi[:,:,tt].T
# Plot the data
iu.plot_image(blos, figsize=(6, 6), title='Blos')

# Crop the data if needed
if crop:
    blos = blos[yorg: yorg + ysize, xorg: xorg + xsize]
    bhor = bhor[yorg: yorg + ysize, xorg: xorg + xsize]
    bazi = bazi[yorg: yorg + ysize, xorg: xorg + xsize]
    if verbose:
        iu.plot_image(blos, figsize=(6, 6), title='Blos after cropping')

if rescale and rescale != 1:
    ysize, xsize = blos.shape 
    ysize = ysize // rescale
    xsize = xsize // rescale
    if ysize % 2 != 0:
        ysize += 1
    if xsize % 2 != 0:
        xsize += 1
    if verbose:
        print(f'Shape before rescale: {blos.shape}')
    # Resample the array to the new size
    blos = resize(blos, (ysize, xsize), anti_aliasing=True)
    bhor = resize(bhor, (ysize, xsize), anti_aliasing=True)
    bazi = resize(bazi, (ysize, xsize), anti_aliasing=True)
    if verbose:
        print(f'Shape after rescale: {blos.shape}')
        iu.plot_image(blos, figsize=(6, 6), title='Blos after rescaling')

# Rotate the data to make it north-up
if not is_north_up:
    blos = iu.make_north_up(blos, fov_angle)
    bhor = iu.make_north_up(bhor, fov_angle)
    bazi = iu.make_north_up(bazi, fov_angle)
    ysize, xsize = blos.shape 
    if verbose:
        print(f'Shape after rotation: {blos.shape}')
    if ysize % 2 != 0 or xsize % 2 != 0:
        print('Resampling the array to ensure that the shape is a multiple of 2')
        print(f'Shape before resampling: {blos.shape}')
        # Resample the array to ensure that the shape is a multiple of 2
        ysize = ysize // 2 * 2
        xsize = xsize // 2 * 2
        blos = resize(blos, (ysize, xsize), anti_aliasing=True)
        bhor = resize(bhor, (ysize, xsize), anti_aliasing=True)
        bazi = resize(bazi, (ysize, xsize), anti_aliasing=True)
        if verbose:
            iu.plot_image(blos, figsize=(6, 6), title='Blos after rotation')


In [ ]:

# Create the mask for the data
data_mask = blos != 0

# Calculate the P, B0 and angular radius of the Sun, and convert the coordinates of center to Heliographic Stonyhurst longitude and latitude
pbr = sun.P(tobs).value, sun.B0(tobs).value, sun.angular_radius(tobs).value
coord = SkyCoord(xcen*u.arcsec, ycen*u.arcsec, obstime=tobs,
                 frame=frames.Helioprojective, observer='earth')
coord_hs = coord.transform_to(frames.HeliographicStonyhurst)
lonlat = coord_hs.lon.value, coord_hs.lat.value
print(f'P, B0, angular radius: {pbr}')
print(f'lon, lat: {lonlat}')

for ii in range(1):
    print(f'Iteration: {ii}')
    # Write the ambig input file
    au.write_ambig_input('.', pix, pbr, lonlat, blos, bhor, bazi)
    # DEBUG: save the variables pix, pbr, lonlat, blos, bhor, bazi to a file in npz format in temp_ambig directory
    np.savez('temp_ambig/ambig_input.npz', pix=pix, pbr=pbr, lonlat=lonlat, blos=blos, bhor=bhor, bazi=bazi)
    # Run the ambig code
    bx, by, bz, bazi, metrics, fig = au.disambig_azimuth(bhor, blos, ambig_par, ambig_executable_path, id, plot_fig=True, save_dir=ambig_save_dir, save_fig=True, data_mask=data_mask)
    # Read the parameters from the ambig par file
    params = au.read_ambig_par_file(ambig_par)
    # Update the seed value
    params['iseed'] = ii
    # # Increase the tfactr value
    # params['tfactr'] *= 1.02
    # # Round the tfactr value to 3 decimal places and set a max value of 0.998
    # params['tfactr'] = min(0.998, round(params['tfactr'], 3))
    au.write_ambig_par_file(ambig_par, params,pprint=True)
    # Clear the output if running in a notebook
    if au.is_notebook():
        clear_output()


In [ ]:
importlib.reload(iu)

In [ ]:
# Plot the results
iu.plot_images([bx, by, bz, bazi], title=['Bx', 'By','Bz', 'Phi'], fontsize=14, figsize=(20, 24), cmap=['gray', 'gray', 'gray', 'twilight'], grid_shape=(2, 2), fig_title=f'ID: {id}', save_fig=True, figname=ambig_save_dir + prefix + 'Bx_By_Bz_Phi.pdf', interpolation='nearest')

In [ ]:
# copy the ambig_par file to the save_dir with suffix appended
ambig_par_copy = ambig_save_dir  + prefix + 'par'
os.system(f'cp {ambig_par} {ambig_par_copy}')
if verbose:
    print(f'Copied {ambig_par} to {ambig_par_copy}')

# copy the ambig_input file to the save_dir with suffix appended
ambig_input_copy = ambig_save_dir + prefix + 'ambig_input.dat'
os.system(f'cp ./ambig_input.dat {ambig_input_copy}')
if verbose:
    print(f'Copied ./ambig_input.dat to {ambig_input_copy}')

In [ ]:
# Save bx, by, bz, bazi to a npz file
np.savez(ambig_save_dir + prefix + 'disambig_output_35_full_res.npz', bx=bx, by=by, bz=bz, bazi=bazi)
print(f'Saved the disambiguated output to {ambig_save_dir + prefix + "disambig_output_35_full_res.npz"}')